# WhisperX joining JSON to SRT

In [36]:
import re,codecs,unidecode
import pandas as pd
import networkx as nx
import matplotlib.pylab as p

import json,glob,os
nMachine=os.uname()[1]
if "iMac" in nMachine:
    nMachine="iMac"
elif "MacBook" in nMachine:
    nMachine="MacBook"
print(nMachine)

iMac


In [37]:
def ding():
    os.system('afplay /System/Library/Sounds/Submarine.aiff')

In [38]:
repJson="/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/%s/Json/"%nMachine
segments=len(glob.glob(repJson+"*.json"))
segments

1

In [39]:
json1=repJson+"extrait-00.json"
with open(json1) as inFile:
    whisper1=json.load(inFile)

### Gestion des timestamps

In [40]:
def second2timestamp(seconds):
    sHours=int(seconds/3600)
    sMinutes=int((seconds-sHours*3600)/60)
    sSeconds=int((seconds-sHours*3600-sMinutes*60))
    sMilliseconds=int((seconds%1)*1000)
    return "%02d:%02d:%02d,%03d"%(sHours,sMinutes,sSeconds,sMilliseconds)

def timestamp2second(timestamp):
    m=re.match("(\d\d):(\d\d):(\d\d),(\d\d\d)",timestamp)
    if m:
        seconds=(int(m.group(1))*60+int(m.group(2)))*60+int(m.group(3))+float(m.group(4))/1000
        return seconds
    else:
        print("problème de timestamp")
        return

def shiftTime(timestamp,decal):
    return second2timestamp(timestamp2second(timestamp)+decal)

In [41]:
punct = list('''!()[]{};:-'"\,<>./?@#$%^&*_~''')
maxSize=8

def removePunct(texte):
    result=texte
    for p in punct:
        result=result.replace(p,"")
    return result

### Gestion des structures JSON de Whisper

In [42]:
def getWords(whisper):
    lWords=[]
    t1=0
    t2=0
    score=0
    for segment in whisper["segments"]:
        for word in segment["words"]:
            w=word["word"]
            if "start" in word:
                t1=second2timestamp(word["start"])
                t2=second2timestamp(word["end"])
                score=word["score"]
            lWords.append((t1,t2,w,score))        
    return [(t1,t2,w,c) for t1,t2,w,c in lWords if w!=""]

def getList(tWhisper):
    return [unidecode.unidecode(removePunct(w)).lower() for _,_,w,_ in tWhisper]

In [43]:
srt=getWords(whisper1)
srtList=getList(srt)

### Trouver la jointure

In [44]:
def findSample(sample,ref):
    lowSample=[t.lower() for t in sample]
    lowRef=[t.lower() for t in ref]
    i=0
    while i<len(ref)-len(sample):
        # print("ref",i)
        if lowSample==lowRef[i:i+len(sample)]:
            return i
        else:
            i+=1
    return None

### Collage des mots
1. on cherche la partie commune entre le nouveau segment et les sous-titres précédents
2. on peut régler la longueur de la partie commune pour optimiser le collage *maxSize*
3. on incrémente les temps de la nouvelle partie à partir du deuxième mot (pour éviter les erreurs de timestamp en début d'enregistrement)

In [45]:
def pasteWords(srt,whisper2):
    srtList=getList(srt)
    w2=getWords(whisper2)
    w2List=getList(w2)
    lenW2=len(w2)
    compSize=min(maxSize,lenW2)
    i=1
    print(compSize)
    while i<lenW2-compSize:
        sample=w2List[i:i+compSize]
        ref=srtList
        # print(sample)
        idx=findSample(sample,ref)
        if idx:
            print("found",i,idx)
            break
        else:
            i+=1
    if idx:
        print("found",i,idx,srtList[idx],srt[idx][0],w2List[i],w2[i][0])
        tDecal=timestamp2second(srt[idx+1][0])-timestamp2second(w2[i+1][0])
        print(second2timestamp(tDecal))
        newSrt=[]
        for n in range(i+1,lenW2):
            nWord=w2[n]
            newWord=(shiftTime(nWord[0],tDecal),shiftTime(nWord[1],tDecal),nWord[2],nWord[3])
            newSrt.append(newWord)
        print(srt[idx+1],newSrt[0])
        return srt[:idx]+newSrt
    else:
        print("problème")
        ding()
        ding()
        ding()
        return None

In [46]:
if segments:
    for i in range(1,segments):
        print(i)
        json2=repJson+"extrait-%02d.json"%i
        with open(json2) as inFile:
            whisper2=json.load(inFile)
        newSrt=pasteWords(srt,whisper2)
        if not newSrt:
            break
        else:
            srt=newSrt
# print(srt)

In [47]:
pause=0.200

srtTours=[]
srtMots=[]
bTour=True
prevEnd=0
pStart=second2timestamp(0)
pEnd=0

for i,(start,end,mot,confidence) in enumerate(srt):
    # print(start,end,mot,confidence)
    tStart=timestamp2second(start)
    tEnd=timestamp2second(end)
    # print(start,tStart,prevEnd)
    if tStart-prevEnd>pause and prevEnd!=0:
        pEnd=second2timestamp(prevEnd)
        pTour=" ".join(srtMots)
        srtTours.append((pStart,pEnd,pTour))
        pStart=start
        prevEnd=tEnd
        srtMots=[mot]
        bPhrase=True
    else:
        srtMots.append(mot)
        prevEnd=tEnd
pTour=" ".join(srtMots)
srtTours.append((pStart,pEnd,pTour))

In [48]:
ding()

In [49]:
srtLines=[]
for i,(start,end,mot) in enumerate(srtTours):
    print(start,end,mot)
    n=i+1
    srtLines.append(str(n))
    srtLines.append(start+" --> "+end)
    srtLines.append(mot)
    srtLines.append("")


00:00:00,000 00:00:17,957 Et c'est Français ! Et c'est de Christian Dominic ! Nous avons bâti la plus belle équipe du monde ! Frederic Bichara qui va retrouver Jean-Dion !
00:00:18,538 00:00:23,260 Jean-Dion, bien promis ! Et c'est Français !
00:00:23,760 00:00:26,701 Quel exploit viennent de réussir les Français !
00:00:36,667 00:00:52,615 Bonsoir, c'était ce d'hier, voici ce d'aujourd'hui. Nous sommes dans les vestiaires de ce Millennium de Cardiff pour ce quarante-deuxième match de Coupe du Monde. Bonsoir Bernard. Bonsoir Christian, bonsoir à tous. Il va falloir battre ce soir les champions du monde en titre.
00:00:52,915 00:00:55,496 C'est pas une messe à faire effectivement, ce sont les champions du monde en titre.
00:00:56,857 00:01:04,962 Entre les deux Coupes du Monde, sur cinquante-deuxième match, ils en ont gagné quarante-six. Donc ça montre bien encore une fois la qualité de cette équipe, mais on vient de voir à travers le reportage que rien n'est impossible.
00:01:05,602 00

In [50]:
with codecs.open(json1.replace(".json","-tours.srt"),"w",encoding="utf8") as outFile:
    for srtLine in srtLines:
        outFile.write(srtLine+"\n")